In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Collect and Explore Data**


In [11]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV data into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/DDP/diabetes.csv')

# Sort the DataFrame by the "Outcome" column in ascending order
df.sort_values(by='Outcome', inplace=True)

# The distribution of the Outcome variable was examined.
df["Outcome"].value_counts()*100/len(df)

# Display the first few rows of the data to get an initial view
print(df.head())

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
383            1       90             62             18       59  25.1   
465            0      124             56             13      105  21.8   
466            0       74             52             10       36  27.8   
467            0       97             64             36      100  36.8   
469            6      154             78             41      140  46.1   

     DiabetesPedigreeFunction  Age  Outcome  
383                     1.268   25        0  
465                     0.452   21        0  
466                     0.269   22        0  
467                     0.600   25        0  
469                     0.571   27        0  


In [7]:
df.index

Int64Index([383, 465, 466, 467, 469, 470, 471, 472, 473, 474,
            ...
            493, 187, 188, 189, 192, 193, 485, 484, 186,   0],
           dtype='int64', length=768)

In [8]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [9]:
df.tail(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
193,11,135,0,0,0,52.3,0.578,40,1
485,0,135,68,42,250,42.3,0.365,24,1
484,0,145,0,0,0,44.2,0.630,31,1
186,8,181,68,36,495,30.1,0.615,60,1
0,6,148,72,35,0,33.6,0.627,50,1


# **Stationary Check**

**Visual Inspection:**

Plot the time series data to visually inspect it for stationarity.
Let's start by visually inspecting the data for any apparent trends or seasonality. We are plotting the time series and look for any noticeable patterns.

In [10]:
# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Outcome'], label='Close Price')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Time Series Data')
plt.grid(True)
plt.legend()
plt.show()

KeyError: 'Close'

<Figure size 1200x600 with 0 Axes>

**Observations:**

If you observe the trends or seasonality in the data, it may indicate non-stationarity.

**Statistical Test (ADF Test):**


The Augmented Dickey-Fuller (ADF) test is a common statistical test to check for stationarity in a time series. It tests the null hypothesis that a unit root is present in the time series data. If the p-value from the test is less than a significance level (e.g., 0.05), you can reject the null hypothesis and consider the data as stationary.

Performing the ADF test on the "Close" price data:

In [ ]:
# !pip install statsmodels

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test
result = adfuller(df['Close'])

# Print the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')


**Notes:** *If the p-value is less than your chosen significance level (e.g., 0.05), you can conclude that the data is stationary. Otherwise, you may need to take further steps to make it stationary, such as differencing or detrending.*


**Observations:**

The ADF Statistic is approximately  0.6868, and the p-value is approximately 0.9895. Based on these results, we can make the following observations:

ADF Statistic: The ADF Statistic is less than the critical values at the 1%, 5%, and 10% significance levels. This suggests that the ADF Statistic is not more negative than the critical values, indicating that the time series data is not strongly stationary.

p-value: The p-value (0.9895) is greater than a common significance level like 0.05. A higher p-value suggests that we fail to reject the null hypothesis, which means the data may not be stationary.

In this case, the ADF test suggests that the data is not strongly stationary. However, the p-value is close to 0.05, indicating marginal evidence against stationarity. It's possible that differencing the data may help in achieving stationarity.

# **Apply Differencing to make the data stationary**

We try differencing the data by subtracting the previous value from the current value and then perform the ADF test again. We repeat this process until we achieve stationarity or reach a reasonable level of differencing. Once we have stationary data, we can proceed with fitting an ARIMA model.

In [ ]:
# Difference the data
df['Close_diff'] = df['Close'].diff()
df = df.dropna()  # Remove the first NaN value

# Perform ADF test on differenced data
result_diff = adfuller(df['Close_diff'])

# Print the results
print('ADF Statistic (Differenced Data):', result_diff[0])
print('p-value (Differenced Data):', result_diff[1])
print('Critical Values:')
for key, value in result_diff[4].items():
    print(f'   {key}: {value}')



**Observations:**

It looks like we've successfully differenced the data, and the results of the Augmented Dickey-Fuller (ADF) test on the differenced data are promising. The ADF Statistic is significantly negative, and the p-value is very close to zero (5.59e-28). This indicates that differencing has likely made our time series data stationary.

# **Finding appropriate model parameters:**


Now that we have stationary data, the next step is to choose the appropriate model parameters for our ARIMA model. To select the values for the p (AR order), d (I order), and q (MA order) parameters, we use the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Create ACF and PACF plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# ACF plot
plot_acf(df['Close_diff'], ax=ax1, lags=40)
ax1.set_title('Autocorrelation Function (ACF)')

# PACF plot
plot_pacf(df['Close_diff'], ax=ax2, lags=40)
ax2.set_title('Partial Autocorrelation Function (PACF)')

plt.show()


**Observations:**

In these plots, we look for significant spikes in the ACF and PACF plots, which indicate the order of the AR and MA components, respectively.
For the ACF plot, the lag where the plot crosses the upper confidence interval for the first time is an indicator of the order q (MA order). In our case, it seems to cross around lag 1 or 2.

For the PACF plot, the lag where the plot crosses the upper confidence interval for the first time is an indicator of the order p (AR order). In our case, it seems to cross around lag 1 or 2 as well.

So, based on the ACF and PACF plots, it suggests that we can start with an ARIMA(2,1,2) model. This means we have an AR component of order 2, a differencing order of 1, and an MA component of order 2. However, we may need to perform some model diagnostics and fine-tuning to find the best model fit.

Now we can proceed to build and fit the ARIMA model to our data.

# **Build model ARIMA(2,1,2)**

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Define the values for p, d, and q
p = 2  # AR order
d = 1  # Differencing order
q = 2  # MA order
from sklearn.preprocessing import MinMaxScaler

# # Assuming df['Close'] contains your time series data
# # Normalize the data using Min-Max scaling
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(df[['Close']])


# Create and fit the ARIMA model
model = ARIMA(df[['Close']], order=(p, d, q))
model_fit = model.fit()

# Display model summary
print(model_fit.summary())

# Plot the residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot(title='Residuals')
plt.show()


**Here's a brief overview of the key results from the model summary:**

Dep. Variable: Name of the dependent variable (Close price).

Model: The model specification, which is ARIMA(2,1,2), indicating an autoregressive order of 2, differencing of order 1, and moving average order of 2.

Log Likelihood: The log-likelihood of the model, which can be used for model comparison.

AIC (Akaike Information Criterion): A measure of model goodness-of-fit. Lower AIC values indicate a better fit.

BIC (Bayesian Information Criterion): Similar to AIC but with a penalty for the number of parameters. It helps avoid overfitting.

Parameters (coef): The estimated coefficients for the AR and MA terms.

sigma2: The estimated variance of the white noise residuals.

Ljung-Box (L1) (Q): A test statistic for assessing the independence of residuals. A low p-value (typically < 0.05) suggests that the residuals are not independent.

Jarque-Bera (JB): A test for assessing the normality of the residuals. A low p-value suggests that the residuals do not follow a normal distribution.

Heteroskedasticity (H): A test for assessing the presence of heteroskedasticity in the residuals. A low p-value suggests that heteroskedasticity is present.

Based on the provided model summary:

The AIC and BIC values are important for model selection. Lower values are generally better, so lower AIC and BIC values suggest a better model fit.

The coefficients (coef) for the AR and MA terms, along with sigma2, are the estimated parameters of the model.

The Ljung-Box test (Q) tests the independence of the residuals, and the Jarque-Bera test (JB) assesses the normality of the residuals.

The Heteroskedasticity test (H) tests for heteroskedasticity in the residuals.

# **Train and Test**

We split our data into training and testing sets. The training set is used to fit the model, and the testing set is used to evaluate its performance. We are using a common split ratio, such as 80% for training and 20% for testing.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (e.g., 80% train, 20% test)
train_size = 0.8
split_index = int(len(df) * train_size)

train_data = df['Close'].iloc[:split_index]
test_data = df['Close'].iloc[split_index:]


**Model Fitting:**

We have already fitted the ARIMA(2,1,2) model to our entire dataset. Now, we can use the training data (train_data) for this purpose.

**Make Predictions:**

In [ ]:
# Make predictions on the test set
start_index = len(train_data)
end_index = start_index + len(test_data) - 1
predictions = model_fit.predict(start=start_index, end=end_index, typ='levels')


**Evaluate Model Performance:**

After making predictions, we can evaluate the model's performance using appropriate evaluation metrics. Common metrics for time series forecasting include Mean Squared Error (MSE), Mean Absolute Error (MAE), and Root Mean Squared Error (RMSE). We can calculate them as follows:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate evaluation metrics
mse = mean_squared_error(test_data, predictions)
mae = mean_absolute_error(test_data, predictions)
rmse = np.sqrt(mse)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')


Mean Squared Error (MSE): The MSE is a measure of the average squared difference between the model's predictions and the actual values. In our case, the MSE is 2110.24. This means, on average, the squared error between the predicted and actual values is 2110.24. Lower MSE values indicate a better fit, but the interpretation depends on the scale of our data.

Mean Absolute Error (MAE): The MAE represents the average absolute difference between the model's predictions and the actual values. With an MAE of 32.91, it indicates that, on average, the model's predictions differ from the actual values by approximately 32.91 units. Lower MAE values are better.

Root Mean Squared Error (RMSE): The RMSE is the square root of the MSE and provides a measure of the average magnitude of the prediction errors. Our RMSE is 45.94. It's in the same units as our data, and lower values indicate a better fit. RMSE is often more interpretable than MSE because it's on the same scale as the data.

# **Result visualization:**

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual and predicted values
plt.figure(figsize=(12, 6))
plt.plot(df.index[split_index:], test_data, label='Actual Close Price', color='blue')
plt.plot(df.index[split_index:], predictions, label='Predicted Close Price', color='red')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Actual vs. Predicted Close Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# from sklearn.model_selection import TimeSeriesSplit


# # Assuming you have a time series dataset loaded into 'data'
# # Replace this with your dataset
# # data = pd.read_csv('your_dataset.csv')


# # Cross-validation to mitigate overfitting
# tscv = TimeSeriesSplit(n_splits=5)
# for train_index, val_index in tscv.split(train_data):
#     train_fold, val_fold = train_data.iloc[train_index], train_data.iloc[val_index]
#     model_fold = ARIMA(train_fold, order=(p, d, q))
#     fitted_fold = model_fold.fit()
#     val_predictions = fitted_fold.forecast(steps=len(val_fold))
#     val_mse = np.mean((val_predictions - val_fold) ** 2)
#     print(f"Validation Mean Squared Error: {val_mse}")


In [ ]:
# from sklearn.model_selection import KFold
# import statsmodels.api as sm

# kf = KFold(n_splits=5)
# rmse_scores = []

# for train_index, test_index in kf.split(train_data):
#     train_df = train_data.iloc[train_index]
#     test_df = train_data.iloc[test_index]

#     model = sm.tsa.ARIMA(train_df, order=(1, 1, 1)).fit()
#     predictions = model.predict(start=len(train_df), end=len(train_data) - 1)
#     rmse = np.sqrt(np.mean((predictions - test_df)**2))
#     rmse_scores.append(rmse)

# print('Average RMSE:', np.mean(rmse_scores))

# Build model ARIMA(1,1,1)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Define the values for p, d, and q
p = 1  # AR order
d = 1  # Differencing order
q = 1  # MA order
from sklearn.preprocessing import MinMaxScaler

# # Assuming df['Close'] contains your time series data
# # Normalize the data using Min-Max scaling
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(df[['Close']])


# Create and fit the ARIMA model
model = ARIMA(df[['Close']], order=(p, d, q))
model_fit = model.fit()

# Display model summary
print(model_fit.summary())

# Plot the residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot(title='Residuals')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (e.g., 80% train, 20% test)
train_size = 0.8
split_index = int(len(df) * train_size)

train_data = df['Close'].iloc[:split_index]
test_data = df['Close'].iloc[split_index:]


In [ ]:
# Make predictions on the test set
start_index = len(train_data)
end_index = start_index + len(test_data) - 1
predictions = model_fit.predict(start=start_index, end=end_index, typ='levels')


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate evaluation metrics
mse = mean_squared_error(test_data, predictions)
mae = mean_absolute_error(test_data, predictions)
rmse = np.sqrt(mse)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')


In [ ]:
import matplotlib.pyplot as plt

# Plot the actual and predicted values
plt.figure(figsize=(12, 6))
plt.plot(df.index[split_index:], test_data, label='Actual Close Price', color='blue')
plt.plot(df.index[split_index:], predictions, label='Predicted Close Price', color='red')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Actual vs. Predicted Close Price')
plt.legend()
plt.grid(True)
plt.show()
